# Main - Impact of Privileged Traders on Market Dynamics 

This notebook runs an experiment on information asymmetry in financial markets, exploring how privileged traders influence market dynamics within the Bornholdt Ising Model framework.

In [ ]:
from source.advantagespinsystem import SpinSystem
from source.utils import read_config_file, reconstruct_grid, visualize_grid, plot_array_list
from datetime import datetime
import numpy as np
from numpy import empty, nan, savetxt
import matplotlib.pyplot as plt

In [15]:
MAX_FILE_SIZE = 100_000

config_filename = "multising.conf"
config = read_config_file(config_filename)

In [16]:
#initialization

#parameters
grid_height   = int(config["grid_height"])
grid_width    = int(config["grid_width"])
alpha         = float(config["alpha"])
j             = float(config["j"])
total_updates = int(config["total_updates"])
init_up       = float(config["init_up"])
T        = float(config["T"])
beta     = 1.0 / T
reduced_alpha              = -2.0 * beta * alpha
reduced_neighbour_coupling = -2.0 * beta * j

#new
privileged_fraction = float(config["privileged_fraction"])
privileged_flip_factor = float(config["privileged_flip_factor"])

magnetisation = empty((min(total_updates, MAX_FILE_SIZE), ), dtype=float)
magnetisation[:] = nan

#init system
spin_system = SpinSystem(
    grid_height=grid_height,
    grid_width=grid_width,
    init_up=init_up,
    privileged_fraction=privileged_fraction,
    privileged_flip_factor=privileged_flip_factor
)

full_grids = []
full_grids.append(reconstruct_grid(spin_system.black, spin_system.white))


In [18]:
for ii in range(total_updates):
    mag = spin_system.update(reduced_neighbour_coupling, reduced_alpha)
    magnetisation[ii % MAX_FILE_SIZE] = mag

    # On peut éventuellement stocker la grille complète après chaque update
    full_grids.append(reconstruct_grid(spin_system.black, spin_system.white))

    # Sauvegarde périodique de la magnétisation
    if ii != 0 and (ii % MAX_FILE_SIZE == 0):
        savetxt(f"magnetisation_{ii}.dat", magnetisation)
        magnetisation[:] = nan

savetxt(f"magnetisation_{ii}.dat", magnetisation)


In [19]:
plot_array_list(full_grids, max_cols=20)

In [ ]:
#bcp moins de polarisation